In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
filenames = glob.glob("csvs/*.csv")

In [3]:
def addUtilitesCost(df, cost):
    df['rent_min'] = df['rent_min'] + cost
    df['rent_max'] = df['rent_max'] + cost

In [4]:
def isLatestScrape(df):
    dateToKeep = df.tail(1)['scraping_date'].values[0]
    timeToKeep = df.tail(1)['scraping_time'].values[0]
    
    isLatestDate = df['scraping_date'] == dateToKeep
    isLatestTime = df['scraping_time'] == timeToKeep
    
    return isLatestDate & isLatestTime

## Calibre

In [5]:
calibre = pd.read_csv(filenames[1])
addUtilitesCost(calibre, 80)

In [6]:
def calibreApts(calibre):
    aptsOfInterest = calibre[isLatestScrape(calibre) & (~calibre['floorplan_name'].str.contains("Hampton")) & 
                             (calibre['rent_min'] < 1750)]
    groupbyCols = aptsOfInterest.columns[:-3].tolist() + [aptsOfInterest.columns[-1]] + [aptsOfInterest.columns[-3]]
    aptsFormatted = aptsOfInterest.groupby(groupbyCols).agg(start_move_in_day=('move_in_day', 'min'),
                                                            end_move_in_day=('move_in_day', 'max'))
    return aptsFormatted

In [7]:
calibreApts(calibre)

start_move_in_day  \
scraping_date scraping_time floorplan_name      bedroom bathroom squareFootage rent_min rent_max move_in_year move_in_month                      
2022/05/22    14            Fillmore Classic II 1.0     1.0      1040.0        1720     1760     2022         5                             22   
                                                                                                              6                              1   

                                                                                                                             end_move_in_day  
scraping_date scraping_time floorplan_name      bedroom bathroom squareFootage rent_min rent_max move_in_year move_in_month                   
2022/05/22    14            Fillmore Classic II 1.0     1.0      1040.0        1720     1760     2022         5                           31  
                                                                                                              6                            1

## Legacy

In [8]:
legacy = pd.read_csv(filenames[0])
addUtilitesCost(legacy, 140)

In [9]:
def legacyApts(legacy):
    aptsOfInterest = legacy[isLatestScrape(legacy) & (legacy['bedroom'] >= 2) & (legacy['rent_min'] < 1750)]
    groupbyCols = aptsOfInterest.columns[:6].tolist() + [aptsOfInterest.columns[-1]] + [aptsOfInterest.columns[-3]]
    aptsFormatted = aptsOfInterest.groupby(groupbyCols).agg(days_range_rent_min_min=('rent_min', 'min'),
                                                            days_range_rent_max_max=('rent_max', 'max'),
                                                            start_move_in_day=('move_in_day', 'min'),
                                                            end_move_in_day=('move_in_day', 'max'))
    return aptsFormatted

In [10]:
legacyApts(legacy).sort_values(['days_range_rent_min_min', 'move_in_month'])

days_range_rent_min_min  \
scraping_date scraping_time floorplan_name bedroom bathroom squareFootage move_in_year move_in_month                            
2022/05/22    14            b1dh           2.0     1.0      980.0         2022         6                                 1520   
                            b1dh_e         2.0     1.0      980.0         2022         5                                 1537   
                                                                                       6                                 1537   
                            b1dh           2.0     1.0      980.0         2022         5                                 1556   
                            b2dh_e         2.0     1.5      1000.0        2022         5                                 1615   
                                                                                       6                                 1615   
                            bt_e           2.0     1.5      1100.0        2022         5                                 1704   
                                                                                       6                                 1704   
                            b2dh_p         2.0     1.5      1000.0        2022         5                                 1715   
                                                                                       6                                 1715   

                                                                                                      days_range_rent_max_max  \
scraping_date scraping_time floorplan_name bedroom bathroom squareFootage move_in_year move_in_month                            
2022/05/22    14            b1dh           2.0     1.0      980.0         2022         6                                 2172   
                            b1dh_e         2.0     1.0      980.0         2022         5                                 2113   
                                                                                       6                                 2196   
                            b1dh           2.0     1.0      980.0         2022         5                                 2092   
                            b2dh_e         2.0     1.5      1000.0        2022         5                                 1965   
                                                                                       6                                 2201   
                            bt_e           2.0     1.5      1100.0        2022         5                                 2100   
                                                                                       6                                 2332   
                            b2dh_p         2.0     1.5      1000.0        2022         5                                 2069   
                                                                                       6                                 2155   

                                                                                                      start_move_in_day  \
scraping_date scraping_time floorplan_name bedroom bathroom squareFootage move_in_year move_in_month                      
2022/05/22    14            b1dh           2.0     1.0      980.0         2022         6                              1   
                            b1dh_e         2.0     1.0      980.0         2022         5                             22   
                                                                                       6                              1   
                            b1dh           2.0     1.0      980.0         2022         5                             22   
                            b2dh_e         2.0     1.5      1000.0        2022         5                             22   
                                                                                       6                              1   
                    

## Madison

In [11]:
madison = pd.read_csv(filenames[2])
addUtilitesCost(madison, 5)

In [12]:
def madisonApts(madison):
    aptsOfInterest = madison[isLatestScrape(madison) & (madison['bedroom'] >= 2) & (madison['rent_min'] < 1750)]
    return aptsOfInterest

In [13]:
madisonApts(madison)

,scraping_date,scraping_time,floorplan_name,bedroom,bathroom,apartment_number,square_footage,rent_min,rent_max,move_in_month,move_in_day,move_in_year
109,2022/05/22,14,Cypress,2,2,2027,987,1713,2193,5,22,2022
